# Gets from Pub Med and process it so it can be chunked, vectorized, and stored in Qdrant

In [43]:
from Bio import Entrez
from lxml import etree
from io import BytesIO

Entrez.email = "charlie.kotula@gmail.com"

# Search query for getting relevant research articles
query = """
(
  rehabilitation OR "physical therapy" OR "return to sport" OR "return to play"
) AND (
  injury OR surgery OR postoperative OR musculoskeletal
) AND (
  exercise OR "therapeutic exercise" OR training
) AND (
  review[pt] OR systematic review[pt] OR meta-analysis[pt]
)
"""

# Get relevant UIDs
handle = Entrez.esearch(
    db='pmc',
    term=query,
    retmax=10, #change
)

uids = Entrez.read(handle)['IdList']
handle.close()

# Functions to extract, clean, and chunk text from PMC
def get_xml(pmc_id):
    handle = Entrez.efetch(
        db='pmc',
        id=pmc_id,
        retmode='xml',
        # rettype='full'
    )

    xml_dat = handle.read()
    return xml_dat

# def parse_xml(xml_doc):


In [45]:
xml_doc = get_xml(uids[5])

xml_tree = etree.parse(BytesIO(xml_doc))

In [47]:
xml_tree.getroot()

<Element pmc-articleset at 0x111c91980>